# **1-2. 설문조사 임베딩 모델 학습**

---
> (주)럽디에서 제공한 설문조사 데이터를 기반으로 문항별 임베딩을 위한 모델을 학습합니다.

> 길이가 매우 가변적인 설문조사의 특성을 고려하여 **Self Attentive Sentence Embedding** 기법을 응용한 모델로 이루어졌으며, 추후 모델의 해석을 위해 Attention을 학습합니다.

> 또한, 주어진 데이터가 약 99.2 : 0.8로 매우 심한 불균형을 가지기 때문에, Siamese Network에 기반한 임베딩 모델 학습을 진행합니다.
---

In [ ]:
# 드라이브 내 Custom Module 및 .ipynb 파일, 그리고 학습 데이터를 저장한 Directory를 입력하세요.
# 설문조사는 총 11개 문항으로 이루어져 있습니다. 학습을 원하는 문항의 번호를 입력하세요.

DIRECTORY = "AI경진대회" # 여기를 변경하세요.
SUVERY_NUMBER = 1

## **(1) 라이브러리 준비**

In [ ]:
# Google Colab을 기반으로 학습을 진행할 경우, BERT 계열의 모델을 사용하기 위해 필요한 라이브러리를 설치합니다.

!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3.0.2
!pip install torch
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'
!pip install torchmetrics

In [ ]:
import torch
import time
import os
import pickle
import random
import pandas as pd
import numpy as np
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import gluonnlp as nlp
import matplotlib.pyplot as plt
from tqdm import tqdm
from kobert_tokenizer import KoBERTTokenizer
from transformers import BertTokenizer, BertModel, AutoModel, AutoTokenizer
from kobert import get_pytorch_kobert_model

## **(2) Drive Mount 및 Custom Module 불러오기**

In [ ]:
# Drive Mount
from google.colab import drive
drive.mount('/content/gdrive')

# Directory 변경
path = "/content/gdrive/My Drive/" + DIRECTORY
os.chdir(path)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
from dataset import SiameseSet
from model import SurveyEmbed, SiameseNet
from train import train_siamese

## **(3) 데이터셋 불러오기**

In [ ]:
# 데이터셋 준비

with open(f'./DATA/2.설문조사/{SUVERY_NUMBER}번.pickle', 'rb') as f:
    data = pickle.load(f)

data.head()

In [ ]:
# Train / Test 분리

index = list(data.index)
random.Random(1398).shuffle(index)
train_idx = index[:(round(len(index) * 0.8))]
test_idx = index[(round(len(index) * 0.8)):]

train = data.iloc[train_idx, :]
test = data.iloc[test_idx, :]

train = train.reset_index(drop=True)
test = test.reset_index(drop=True)

## **(4) KcELECTRA, Tokenizer 불러오기**

In [ ]:
# KcELECTRA, Tokenizer

tokenizer = AutoTokenizer.from_pretrained("beomi/KcELECTRA-base-v2022")
kc_model = AutoModel.from_pretrained("beomi/KcELECTRA-base-v2022")

## **(5) DataLoader 준비**

In [ ]:
# Dataset

train_set = SiameseSet(tokenizer=tokenizer, text_column=list(data.columns)[0], data=train, n_pairing=100, shuffle=False)
valid_set = SiameseSet(tokenizer=tokenizer, text_column=list(data.columns)[0], data=test, n_pairing=100, shuffle=False)

In [ ]:
# DataLoader

BATCH_SIZE=64

train_loader = torch.utils.data.DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=False, drop_last=True)
valid_loader = torch.utils.data.DataLoader(valid_set, batch_size=BATCH_SIZE, shuffle=False, drop_last=True)

## **(6) 모델 생성**

In [ ]:
# Model (Siamese Network)

model = SiameseNet(num_layers=2,
                   embedding_size=768,
                   hidden_size=384,
                   attention_dim=512,
                   keys=30,
                   batch_size=BATCH_SIZE)  

## **(7) 모델 학습**

In [ ]:
# Train Model

model_aft, train_loss, valid_loss = train_siamese(train_dataloader=train_loader,
                                                  valid_dataloader=valid_loader,
                                                  valid=True,
                                                  model=model,
                                                  epochs=20,
                                                  optimizer=optim.Adam,
                                                  criterion=ContrastiveLoss,
                                                  scheduler=optim.lr_scheduler.StepLR,
                                                  gamma=0.8,
                                                  step_size=4,
                                                  lr=8e-03,
                                                  coef=0.1,
                                                  margin=1.0,
                                                  SURVEY_NUMBER=SURVEY_NUMBER,
                                                  device=device)

## **(8) 학습 결과 시각화**

In [ ]:
# Accuracy 시각화

plt.figure(figsize=(12, 6))
plt.plot(train_loss)
plt.plot(valid_loss)
plt.title('Loss History')
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.grid(True)

plt.tight_layout()
plt.show()